In [63]:
%load_ext autoreload
%autoreload 2
from matchviz import fetch_all_matches
import polars as pl
from matchviz import get_tile_coords, load_points, ome_ngff_to_coords, plot_points, scale_points, translate_points, read_bigstitcher_xml, tokenize, summarize_match, summarize_matches


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:

bs_url = "s3://aind-open-data/exaSPIM_708373_2024-04-02_19-49-38_alignment_2024-05-07_18-15-25/bigstitcher.xml"
spimdata = read_bigstitcher_xml(bs_url)
tile_coords= get_tile_coords(spimdata)
url = "s3://aind-open-data/exaSPIM_708373_2024-04-02_19-49-38_alignment_2024-05-07_18-15-25/interestpoints.n5/tpId_0_viewSetupId_0/beads/"
image_url = 's3://aind-open-data/exaSPIM_708373_2024-04-02_19-49-38/SPIM.ome.zarr/tile_x_0000_y_0000_z_0000_ch_488.zarr'

points, matches = load_points(url)
coords = ome_ngff_to_coords(image_url)
points_tx = scale_points(translate_points(points, coords), coords)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
matches_dict = fetch_all_matches('s3://aind-open-data/exaSPIM_708373_2024-04-02_19-49-38_alignment_2024-05-07_18-15-25/interestpoints.n5/')

In [48]:
tuple(matches_dict.keys())[0]

's3://aind-open-data/exaSPIM_708373_2024-04-02_19-49-38_alignment_2024-05-07_18-15-25/interestpoints.n5/tpId_0_viewSetupId_14/beads'

In [44]:
dframes = {k: summarize_match(v) for k,v in matches_dict.items()}

In [147]:
from matchviz import summarize_match


match_vals = tuple(matches_dict.values())
summarized_matches = tuple(map(summarize_match, match_vals))

In [66]:

df = summarize_matches(spimdata, matches_dict)

In [78]:
df[0]['tile_coords_tokenized']

tile_coords_tokenized
list[i64]
"[4, 1]"


In [82]:
df.with_columns(
    pl.col('tile_coords_tokenized').map_elements(lambda v: str(v.to_list()))).select(
        ['tile_coords_tokenized', 'bs_image_name', 'id_self', 'id_other', 'num_matches']
    ).write_csv('foo.csv')

In [ ]:
pl.datatypes.